In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import json
import pickle
from datetime import date, timedelta

In [ ]:
_kamers = open("kamers.json", "r")
kamers = json.loads(_kamers.read())
_kamers.close()

_devices = open("devices.p", "rb")
devices = pickle.load(_devices)
_devices.close()

In [ ]:
one = devices['Huis']['Electriciteit_opwek_dak']
# The monitoring device I'm using _always_ reads either 0.1 or 0.2 Wh / 15 minutes
# , so only retain values above that amount
one = one[one.apply(lambda x: x['value'] > 0.2, axis=1)]

two = devices['Huis']['Electriciteit_opwek_garage']

cm_low_in = devices['Huis']['Electriciteit_gebruik_laag']
cm_high_in = devices['Huis']['Electriciteit_gebruik_hoog']

cm_low_out = devices['Huis']['Electriciteit_teruglevering_laag']
cm_high_out = devices['Huis']['Electriciteit_teruglevering_hoog']


def merge(a, b):
    c = pd.merge(a, b, left_index=True, right_index=True).sum(axis=1).to_frame()
    return c.rename(columns={c.columns[0]: 'value' })

prod = merge(one, two)
usage = merge(cm_low_in, cm_high_in)
feedback = merge(cm_low_out, cm_high_out)

# Todo: derive from the dataset
years = ['2018', '2019', '2020']

sns.set(rc={'figure.figsize':(18, 6)})

In [ ]:
# From https://stackoverflow.com/a/41259922
def align_yaxis(ax1, ax2):
    """Align zeros of the two axes, zooming them out by same ratio"""
    axes = (ax1, ax2)
    extrema = [ax.get_ylim() for ax in axes]
    tops = [extr[1] / (extr[1] - extr[0]) for extr in extrema]
    # Ensure that plots (intervals) are ordered bottom to top:
    if tops[0] > tops[1]:
        axes, extrema, tops = [list(reversed(l)) for l in (axes, extrema, tops)]

    # How much would the plot overflow if we kept current zoom levels?
    tot_span = tops[1] + 1 - tops[0]

    b_new_t = extrema[0][0] + tot_span * (extrema[0][1] - extrema[0][0])
    t_new_b = extrema[1][1] - tot_span * (extrema[1][1] - extrema[1][0])
    axes[0].set_ylim(extrema[0][0], b_new_t)
    axes[1].set_ylim(t_new_b, extrema[1][1])

def chart_timespan(begin, end, resample, label):
    begin = begin.strftime('%Y-%m-%d')
    end = end.strftime('%Y-%m-%d')
    
    df = usage[begin:end].index.to_frame().drop(['_from'], axis=1)
    df['Productie'] = prod[begin:end]['value'].apply(lambda x: x*-1)
    df['Consumptie'] = usage[begin:end]['value']
    df['Teruglevering'] = feedback[begin:end]['value'].apply(lambda x: x*-1)
    df = df.fillna(0)
    df['Netto gebruik'] = df.apply(lambda r: r['Consumptie'] - ((r['Productie']) - r['Teruglevering']), axis=1)
    
    #rt = merge(usage, feedback)[begin:end].cumsum(axis=0).apply(lambda x: x / 1000)
    rt = merge(df['Consumptie'], df['Teruglevering'])[begin:end].cumsum(axis=0).apply(lambda x: x / 1000)
    
    df = resample(df)
    rt = resample(rt)

    details = df.plot.area(stacked = False)
    details.set_xlabel('Tijd van ' + begin + ' tot ' + end)
    details.set_ylabel(label)
    
    running_total = rt['value'].plot(ax=details, style='r-', secondary_y=True, label='Running total')
    running_total.set_ylabel('Running total in kWh')
    running_total.legend(bbox_to_anchor=(0.12, 0.98))
        
    return align_yaxis(details, running_total)    

In [ ]:
today = date.today()
yesterday = today - timedelta(days=1)
week = today - timedelta(days=7)
month = today - timedelta(days=30)
start_of_year = date(date.today().year, 1, 1)

chart_timespan(yesterday, today, lambda df: df, 'Wh / 15 minuten')
chart_timespan(yesterday, today, lambda df: df.resample('H').sum(), 'Wh')
chart_timespan(week, today, lambda df: df.resample('H').sum(), 'Wh')
chart_timespan(month, today, lambda df: df.resample('D').sum(), 'Wh')
for year in years[1:]:
    a = date(int(year), 1, 1)
    b = date(int(year), 12, 31)
    chart_timespan(a, b, lambda df: df.resample('W').sum().apply(lambda x: x / 1000), 'kWh / week')

In [ ]:
def compare_years(source, years, x_axis = lambda d: d.index.dayofyear):
    fig, ax = plt.subplots()
    for year in years:
        x = source.loc[year, 'value']
        x = x.map(lambda x: x / 1000, na_action=None)
        ax.plot(x_axis(x), x, marker='x', linestyle='-', linewidth=0.5, label='kWh')
    ax.set_ylabel('kWh')
    ax.legend()

def compare_years_cumsum(source, years, xlabel):
    fig, ax = plt.subplots()
    source = source.fillna(0)
    for year in years:
        x = source.loc[year]
        x = x.cumsum(axis=0)
        x = x.map(lambda x: x / 1000, na_action=None)
        lbl = year + ' end: ' + str(round(x.iloc[-1], 2)) + ' kWh'
        ax.plot(x.index.dayofyear, x, marker='.', linestyle='-', linewidth=0.5, label=lbl)
    ax.set_ylabel('kWh')
    ax.set_xlabel(xlabel)
    ax.legend()

In [ ]:
compare_years_cumsum(one['value'], years, 'Huis')
# The first 2 years there was no data anyways
compare_years_cumsum(two['value'], years[2:], 'Garage')
compare_years_cumsum(prod['value'], years, 'Beide')

In [ ]:
def scatter_plot(df):
    df['day'] = df.index.dayofyear
    df['hour'] = df.index.hour
    df = df[df.apply(lambda x: x['value'] > 10, axis=1)]
    df.plot.scatter(x='day', y='hour', c='value', cmap='viridis')

In [ ]:
scatter_plot(one)

In [ ]:
scatter_plot(two)

In [ ]:
scatter_plot(prod)